In [20]:
import pandas as pd
import tensorflow as tf

In [21]:
csv = pd.read_csv('2017_ATP_rank_overview_match.csv')
csv.head(3)

,player_id,Aces,Double Faults,1st Serve,1st Serve Points Won,2nd Serve Points Won,Service Points Won,Break Points Saved,1st Serve Return Points Won,2nd Serve Return Points Won,...,height_ft,height_inches,height_cm,handedness,backhand,year,rank,age,point_per_played,player_slug_y
0,n409,0.137931,0.00,0.726190,0.639344,0.608696,0.630952,0.5,0.193548,0.542857,...,"6'1""",73.0,185.0,Left-Handed,Two-Handed Backhand,2017.0,1.0,31.0,591.388889,rafael-nadal
1,n552,0.160000,0.00,0.710145,0.734694,0.450000,0.652174,0.4,0.211538,0.520000,...,"5'10""",70.0,178.0,Right-Handed,Two-Handed Backhand,2017.0,22.0,27.0,99.210526,kei-nishikori
2,r975,0.200000,0.15,0.459016,0.857143,0.484848,0.655738,0.5,0.250000,0.181818,...,"6'5""",77.0,196.0,Right-Handed,Two-Handed Backhand,2017.0,24.0,26.0,89.750000,milos-raonic


In [22]:
ft = csv['height_ft'].values
lst = []
for i in ft:
    sp = i.split("'")
    lst.append(round(int(sp[0])*30.48 + int(sp[1][:-1])*2.54,2))

csv['height_ft'] = lst

In [23]:
import matplotlib.pyplot as plt
import numpy as np
# import seaborn as sns

# heat = csv.corr()
# heat['Result'].plot.bar()

In [24]:
# sns.heatmap(heat.corr())

In [25]:
# heat.corr()

In [26]:
csv.columns

Index(['player_id', 'Aces', 'Double Faults', '1st Serve',
       '1st Serve Points Won', '2nd Serve Points Won', 'Service Points Won',
       'Break Points Saved', '1st Serve Return Points Won',
       '2nd Serve Return Points Won', 'Break Points Coverted',
       'Return Points Won', 'Result', 'player_slug_x', 'flag_code',
       'birth_year', 'birth_month', 'birth_day', 'turned_pro', 'weight_lbs',
       'weight_kg', 'height_ft', 'height_inches', 'height_cm', 'handedness',
       'backhand', 'year', 'rank', 'age', 'point_per_played', 'player_slug_y'],
      dtype='object')

In [27]:
def plot_loss(history):
    plt.plot(history.history['loss'])
    plt.title('Model Loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Test'], loc=0)
    
def plot_acc(history):
    plt.plot(history.history['accuracy'])
    plt.title('Model accuracy')
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Test'], loc=0)

In [49]:
X = csv
X = X.drop('player_id', axis=1)
X = X.drop('1st Serve', axis=1)
X = X.drop('Result', axis=1)
X = X.drop('player_slug_x', axis=1)
X = X.drop('flag_code', axis=1)
X = X.drop('birth_year', axis=1)
X = X.drop('birth_month', axis=1)
X = X.drop('birth_day', axis=1)
X = X.drop('turned_pro', axis=1)
X = X.drop('handedness', axis=1)
X = X.drop('backhand', axis=1)
X = X.drop('year', axis=1)
X = X.drop('player_slug_y', axis=1)
X = X.astype('float32')
X.shape

(6967, 18)

In [50]:
from tensorflow.keras import utils

y = csv['Result']
y = utils.to_categorical(y,2)
y.shape

(6967, 2)

In [53]:
X['Result'] = csv['Result']
X.to_csv('ATP_ML.csv', index=False)

In [ ]:
# from sklearn.model_selection import train_test_split

# X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.3, random_state=1004)

# X_train.shape, X_test.shape, y_train.shape, y_test.shape
# 스플릿된 데이터가 0과 1을 고르게 가지고 있는지 확인
np.unique(y_train, return_counts=True), np.unique(y_test, return_counts=True) # 데이터가 골고루 나누어져 있음

((array([0., 1.], dtype=float32), array([4876, 4876], dtype=int64)),
 (array([0., 1.], dtype=float32), array([2091, 2091], dtype=int64)))

In [ ]:
from tensorflow.keras import layers, models
from tensorflow.keras import optimizers

rslt = []

def model(my_lr, epoch, batch):
    adam = optimizers.Adam(my_lr)
    model = models.Sequential()
    model.add(layers.Dense(128, input_shape=(X_train.shape[1],), activation='relu'))
    model.add(layers.Dense(128, activation='relu'))
    model.add(layers.Dense(128, activation='relu'))
    model.add(layers.Dense(2,  activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])
    history = model.fit(X_train, y_train, epochs=epoch, verbose=False, batch_size=batch)
#     plot_loss(history)
#     plt.show()
#     plot_acc(history)
#     plt.show()
    
    score = model.evaluate(X_test, y_test, verbose=0)
#     print('테스트 손실값:', score[0])
#     print('테스트 정확도:', score[1])
    
    rslt.append([my_lr,epoch,batch,history.history['accuracy'][-1],score[1]])
    
if __name__ == '__main__':
    lrs = [0.00001,0.00005,0.0001]
    epochs = [30,60,90,120,150,180,210,240,270,300]
    batchs = [100,200,300,400,500]
    
    for i in lrs:
        for j in epochs:
            for k in batchs:
                model(i,j,k)
                
pd.DataFrame(rslt, columns=['학습률','반복횟수','배치','훈련데이터정확도','테스트데이터정확도'])

KeyboardInterrupt: 